<a href="https://colab.research.google.com/github/GenAIUnplugged/langchain_series/blob/main/02_chat_model_basic_conversation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain-core langchain langchain-community langchain-openai

In [ ]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage, HumanMessage, AIMessage

In [ ]:
messages = [
    ("system","You are an expert historian"),
    ("user","Tell me about {topic}. Provide a bulleted list of key points")
]

**Steps to invoke the model with ChatPromptTemplate**
1. Format the Prompt
You need to format the prompt by passing a dictionary with the value for the {topic} placeholder.

2. Call the LLM with the formatted prompt

In [ ]:
prompt = ChatPromptTemplate.from_messages(messages)
formatted_prompt = prompt.invoke({"topic":"Aryan race"})

**In the below code, we use the max_tokens to limit the number of maximum tokens**

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini",temperature=0,max_tokens=500)

In [ ]:
result = llm.invoke(formatted_prompt)
result

AIMessage(content='The concept of the "Aryan race" has a complex and controversial history. Here are some key points:\n\n- **Origins of the Term**: The term "Aryan" originally referred to a group of ancient peoples who spoke Indo-Iranian languages. It is derived from the Sanskrit word "ārya," meaning "noble."\n\n- **Linguistic Classification**: In the 19th century, linguists used "Aryan" to describe a branch of the Indo-European language family, which includes languages such as Sanskrit, Persian, and various European languages.\n\n- **Racial Theories**: In the late 19th and early 20th centuries, the term was misappropriated by racial theorists who posited that the "Aryan race" was a superior racial group, often associated with Northern Europeans.\n\n- **Nazi Ideology**: The concept was co-opted by the Nazi regime in Germany, which promoted the idea of a "master race" of Aryans, characterized by specific physical traits (e.g., light skin, blonde hair, blue eyes). This ideology justified

In [ ]:
from langchain_core.output_parsers import StrOutputParser

This section of the code demonstrates how to create a simple chain in LangChain, process a request, and parse the output.

model = prompt | llm | StrOutputParser()

This line creates a LangChain chain. In LangChain, a chain is a sequence of components that process data. Here, we have three components linked together using the | operator:

**prompt**: This is the ChatPromptTemplate we defined earlier. It prepares the input for the language model by formatting the user's query with a system message.\
**llm**: This is the ChatOpenAI instance, which is our language model. It takes the formatted prompt as input and generates a response.\
**StrOutputParser()**: This is an output parser. It takes the raw output from the language model and converts it into a more usable format. In this case, StrOutputParser simply converts the language model's output into a string [1].

In [ ]:
model = prompt | llm | StrOutputParser()
result = model.invoke({"topic":"Adolf Hitler"})
result

'Certainly! Here’s a bulleted list of key points about Adolf Hitler:\n\n- **Birth and Early Life**: \n  - Born on April 20, 1889, in Braunau am Inn, Austria-Hungary (now Austria).\n  - Moved to Linz and later to Vienna, where he struggled as an artist and developed his nationalist and anti-Semitic views.\n\n- **World War I**: \n  - Served as a soldier in the German Army during World War I.\n  - Awarded the Iron Cross for bravery but was wounded twice.\n\n- **Political Rise**: \n  - Joined the German Workers\' Party (DAP) in 1919, which later became the National Socialist German Workers\' Party (NSDAP or Nazi Party).\n  - Became the party\'s leader in 1921 and developed a platform of extreme nationalism, anti-Semitism, and anti-communism.\n\n- **Mein Kampf**: \n  - Wrote "Mein Kampf" (My Struggle) while imprisoned in 1924, outlining his ideology and future plans for Germany.\n\n- **Chancellorship**: \n  - Appointed Chancellor of Germany on January 30, 1933.\n  - Quickly consolidated pow

In [ ]:
# prompt: provide an example using the RunnableLambda. Use a different example

from langchain_core.runnables import RunnableLambda

# Define a simple function that takes input and adds a greeting
def add_greeting(input_text):
  """Adds a greeting to the input text."""
  return f"Hello, {input_text}!"

# Create a RunnableLambda from the function
greeting_chain = RunnableLambda(add_greeting)

# Invoke the RunnableLambda with some input
result = greeting_chain.invoke("World")
print(result)

# You can also chain RunnableLambda with other components
def uppercase_text(input_text):
  """Converts the input text to uppercase."""
  return input_text.upper()

uppercase_chain = RunnableLambda(uppercase_text)

# Chain the greeting and uppercase functions
full_chain = greeting_chain | uppercase_chain

# Invoke the chained runnables
result_chained = full_chain.invoke("langchain")
result_chained


Hello, World!


'HELLO, LANGCHAIN!'

**This section of the code demonstrates a more advanced way to build a processing pipeline using LangChain's Runnable components.**



1.   RunnableLambda allows you to wrap a function or a lambda so it can be used within a LangChain Runnable chain.
2.   RunnableSequence allows you to define a sequence of Runnable objects to be executed in order.



**format_prompt:** This RunnableLambda takes an input x (which is expected to be a dictionary) and uses it to format the prompt_template. The x syntax unpacks the dictionary x as keyword arguments to the format_prompt method \
**invoke_model:** This RunnableLambda takes the formatted prompt as input x (which is expected to be a PromptValue object) and invokes the language model (llm) with the prompt converted to a list of messages using x.to_messages().\
**parser_output:** This RunnableLambda takes the output from the language model as input x (which is expected to be a Message object) and extracts the content of the message using x.content.

In [ ]:
from langchain_core.runnables import RunnableLambda,RunnableSequence
from IPython.display import Markdown

messages = [
    ("system","You are an comedian who tells jokes about a {topic}"),
    ("human","Tell me {joke_count} jokes")
]

prompt_template = ChatPromptTemplate.from_messages(messages)

format_prompt = RunnableLambda(lambda x:prompt_template.format_prompt(**x))
invoke_model = RunnableLambda(lambda x:llm.invoke(x.to_messages()))
parser_output = RunnableLambda(lambda x:x.content)

chain = format_prompt | invoke_model | parser_output
response = chain.invoke({"topic":"lawyers","joke_count":5})
Markdown(response)

#Seeing Intermediate Steps in a LangChain Runnable Chain
# async def run_and_log():
#     async for step in chain.astream_log({"topic":"lawyers","joke_count":5}):
#         print("\n--- Intermediate Step ---")
#         print(step)
#         print("-------------------------\n")

# # Run the asynchronous function
# await run_and_log()


--- Intermediate Step ---
RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '4c58ea63-130d-4b0c-ba92-973ad476a908',
            'logs': {},
            'name': 'RunnableSequence',
            'streamed_output': [],
            'type': 'chain'}})
-------------------------


--- Intermediate Step ---
RunLogPatch({'op': 'add',
  'path': '/logs/RunnableLambda',
  'value': {'end_time': None,
            'final_output': None,
            'id': '8c0c95b9-a201-4744-97d1-deef8ad0d87b',
            'metadata': {},
            'name': 'RunnableLambda',
            'start_time': '2025-05-10T14:53:39.202+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': ['seq:step:1'],
            'type': 'chain'}})
-------------------------


--- Intermediate Step ---
RunLogPatch({'op': 'add',
  'path': '/logs/RunnableLambda/streamed_output/-',
  'value': ChatPromptValue(messages=[SystemMessage(content='You are an co